# Fase 3: Ground Truth Generation con Campionamento Stratificato per VIN

Questo notebook genera il ground truth utilizzando un **campionamento stratificato basato su VIN** per garantire overlap sufficiente.

## Problema Risolto
Il campionamento casuale generava solo 4 VIN comuni (0.007%), insufficienti per l'evaluation.

## Soluzione: VIN-Stratified Sampling
1. Caricare dataset completi
2. Identificare TUTTI i VIN comuni
3. Campionare preferenzialmente record con VIN comuni
4. Target: **50-100 match pairs** (0.1-0.2% overlap)
5. Riempire con campioni casuali
6. Generare ground truth sui campioni stratificati

## Obiettivi
- **Craigslist**: 10,000 record (inclusi record con VIN comuni)
- **UsedCars**: 50,000 record (inclusi record con VIN comuni)
- **Match pair target**: 50-100 coppie
- **Split**: 70% train, 10% validation, 20% test

## 1. Import Librerie

In [ ]:
import sys
import os

# Aggiungi il percorso della cartella src al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
sys.path.append(os.path.abspath('..'))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import moduli personalizzati
from src.ground_truth import GroundTruthGenerator

# Imposta stile grafici
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Librerie importate con successo!")

## 2. Caricamento Dataset Completi

In [ ]:
# Percorsi dei dataset allineati
craigslist_path = '../data/processed/craigslist_aligned.csv'
usedcars_path = '../data/processed/usedcars_aligned.csv'

# Carica i dataset COMPLETI
print("Caricamento dataset completi...")
df_craigslist_full = pd.read_csv(craigslist_path, low_memory=False)
print(f"✓ Craigslist: {len(df_craigslist_full):,} righe, {len(df_craigslist_full.columns)} colonne")

df_usedcars_full = pd.read_csv(usedcars_path, low_memory=False)
print(f"✓ US Used Cars: {len(df_usedcars_full):,} righe, {len(df_usedcars_full.columns)} colonne")

## 3. Identificazione VIN Comuni nei Dataset Completi

In [ ]:
# Filtra VIN validi (lunghezza >= 11)
print("Analisi VIN nei dataset completi...")

# Craigslist VIN validi
craigslist_valid_vin = df_craigslist_full[
    df_craigslist_full['vin'].notna() & 
    (df_craigslist_full['vin'].str.len() >= 11)
].copy()

# UsedCars VIN validi
usedcars_valid_vin = df_usedcars_full[
    df_usedcars_full['vin'].notna() & 
    (df_usedcars_full['vin'].str.len() >= 11)
].copy()

print(f"\nCraigslist VIN validi: {len(craigslist_valid_vin):,} / {len(df_craigslist_full):,}")
print(f"UsedCars VIN validi: {len(usedcars_valid_vin):,} / {len(df_usedcars_full):,}")

In [ ]:
# Trova VIN comuni
vins_craigslist = set(craigslist_valid_vin['vin'].unique())
vins_usedcars = set(usedcars_valid_vin['vin'].unique())

common_vins = vins_craigslist & vins_usedcars

print("\nVIN COMUNI NEI DATASET COMPLETI:")
print("=" * 80)
print(f"VIN unici in Craigslist: {len(vins_craigslist):,}")
print(f"VIN unici in UsedCars: {len(vins_usedcars):,}")
print(f"VIN comuni: {len(common_vins):,}")

if len(common_vins) == 0:
    print("\n⚠️ ATTENZIONE: Nessun VIN comune trovato!")
    print("Il campionamento stratificato non può essere applicato.")
else:
    print(f"\n✓ Trovati {len(common_vins):,} VIN comuni per il campionamento stratificato")

## 4. Campionamento Stratificato per VIN

Strategia:
1. Selezionare TUTTI i record con VIN comuni
2. Se superano la dimensione target, campionare casualmente
3. Riempire con record casuali fino alla dimensione desiderata

In [ ]:
# Parametri campionamento
SAMPLE_SIZE_CRAIGSLIST = 10000
SAMPLE_SIZE_USEDCARS = 50000
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

print("\nCAMPIONAMENTO STRATIFICATO PER VIN:")
print("=" * 80)

In [ ]:
# ========== CRAIGSLIST STRATIFIED SAMPLING ==========

# Step 1: Record con VIN comuni
craigslist_with_common_vin = craigslist_valid_vin[
    craigslist_valid_vin['vin'].isin(common_vins)
].copy()

print(f"\nCraigslist:")
print(f"  Record con VIN comuni: {len(craigslist_with_common_vin):,}")

# Step 2: Campiona o prendi tutti
if len(craigslist_with_common_vin) >= SAMPLE_SIZE_CRAIGSLIST:
    # Troppi record con VIN comuni, campiona
    df_craigslist = craigslist_with_common_vin.sample(
        n=SAMPLE_SIZE_CRAIGSLIST, 
        random_state=RANDOM_SEED
    )
    print(f"  → Campionati {SAMPLE_SIZE_CRAIGSLIST:,} da record con VIN comuni")
else:
    # Prendi tutti i record con VIN comuni
    df_craigslist = craigslist_with_common_vin.copy()
    
    # Step 3: Riempi con record casuali
    remaining_size = SAMPLE_SIZE_CRAIGSLIST - len(df_craigslist)
    
    if remaining_size > 0:
        # Record senza VIN comuni
        craigslist_without_common_vin = df_craigslist_full[
            ~df_craigslist_full.index.isin(craigslist_with_common_vin.index)
        ]
        
        # Campiona record casuali
        random_sample = craigslist_without_common_vin.sample(
            n=min(remaining_size, len(craigslist_without_common_vin)),
            random_state=RANDOM_SEED
        )
        
        df_craigslist = pd.concat([df_craigslist, random_sample], ignore_index=False)
        
        print(f"  → Presi tutti {len(craigslist_with_common_vin):,} record con VIN comuni")
        print(f"  → Aggiunti {len(random_sample):,} record casuali")

print(f"  ✓ Totale Craigslist sample: {len(df_craigslist):,}")

In [ ]:
# ========== USEDCARS STRATIFIED SAMPLING ==========

# Step 1: Record con VIN comuni
usedcars_with_common_vin = usedcars_valid_vin[
    usedcars_valid_vin['vin'].isin(common_vins)
].copy()

print(f"\nUsedCars:")
print(f"  Record con VIN comuni: {len(usedcars_with_common_vin):,}")

# Step 2: Campiona o prendi tutti
if len(usedcars_with_common_vin) >= SAMPLE_SIZE_USEDCARS:
    # Troppi record con VIN comuni, campiona
    df_usedcars = usedcars_with_common_vin.sample(
        n=SAMPLE_SIZE_USEDCARS, 
        random_state=RANDOM_SEED
    )
    print(f"  → Campionati {SAMPLE_SIZE_USEDCARS:,} da record con VIN comuni")
else:
    # Prendi tutti i record con VIN comuni
    df_usedcars = usedcars_with_common_vin.copy()
    
    # Step 3: Riempi con record casuali
    remaining_size = SAMPLE_SIZE_USEDCARS - len(df_usedcars)
    
    if remaining_size > 0:
        # Record senza VIN comuni
        usedcars_without_common_vin = df_usedcars_full[
            ~df_usedcars_full.index.isin(usedcars_with_common_vin.index)
        ]
        
        # Campiona record casuali
        random_sample = usedcars_without_common_vin.sample(
            n=min(remaining_size, len(usedcars_without_common_vin)),
            random_state=RANDOM_SEED
        )
        
        df_usedcars = pd.concat([df_usedcars, random_sample], ignore_index=False)
        
        print(f"  → Presi tutti {len(usedcars_with_common_vin):,} record con VIN comuni")
        print(f"  → Aggiunti {len(random_sample):,} record casuali")

print(f"  ✓ Totale UsedCars sample: {len(df_usedcars):,}")

In [ ]:
# Verifica VIN comuni nel sample
sample_vins_craigslist = set(df_craigslist[df_craigslist['vin'].notna()]['vin'])
sample_vins_usedcars = set(df_usedcars[df_usedcars['vin'].notna()]['vin'])
sample_common_vins = sample_vins_craigslist & sample_vins_usedcars

print(f"\nVIN COMUNI NEL CAMPIONE:")
print("=" * 80)
print(f"VIN comuni: {len(sample_common_vins):,}")
print(f"Match pairs attesi: ~{len(sample_common_vins):,}")
print(f"Percentuale overlap: {len(sample_common_vins) / (SAMPLE_SIZE_CRAIGSLIST + SAMPLE_SIZE_USEDCARS) * 100:.3f}%")

if len(sample_common_vins) < 50:
    print(f"\n⚠️ ATTENZIONE: Overlap insufficiente ({len(sample_common_vins)} < 50)")
    print(f"Consiglio: aumentare dimensione campione o verificare VIN comuni disponibili")
else:
    print(f"\n✓ Overlap sufficiente: {len(sample_common_vins):,} VIN comuni")

## 5. Reset Indici e Generazione record_id

IMPORTANTE: Reset indici per evitare problemi con il GroundTruthGenerator

In [ ]:
# Reset indici per avere indici sequenziali 0, 1, 2, ...
print("\nReset indici e generazione record_id...")

df_craigslist = df_craigslist.reset_index(drop=True)
df_usedcars = df_usedcars.reset_index(drop=True)

# Genera record_id in formato df1_XXX e df2_XXX
# Questo DEVE corrispondere a ciò che GroundTruthGenerator fa internamente
df_craigslist['record_id'] = ['df1_' + str(i) for i in range(len(df_craigslist))]
df_usedcars['record_id'] = ['df2_' + str(i) for i in range(len(df_usedcars))]

print(f"✓ Indici resettati")
print(f"✓ record_id generati: df1_0 ... df1_{len(df_craigslist)-1}")
print(f"✓ record_id generati: df2_0 ... df2_{len(df_usedcars)-1}")

## 6. Generazione Ground Truth

In [ ]:
# Crea generatore di ground truth
print("\nCreazione generatore ground truth...")
gt_generator = GroundTruthGenerator(
    df1=df_craigslist,
    df2=df_usedcars,
    vin_column='vin',
    min_vin_length=11
)
print("✓ Generatore creato")

In [ ]:
# Trova match basati su VIN
print("\nRicerca match basati su VIN...")
matches = gt_generator.find_matches()

print(f"\n✓ Match trovati: {len(matches):,}")
print("\nCampione di match:")
display(matches.head(10))

In [ ]:
# Genera non-match
print("\nGenerazione non-match...")
non_matches = gt_generator.generate_non_matches(
    ratio=1.0  # Stesso numero di match (ratio 1:1)
)

print(f"\n✓ Non-match generati: {len(non_matches):,}")
print("\nCampione di non-match:")
display(non_matches.head(10))

In [ ]:
# Crea ground truth completo
ground_truth = gt_generator.create_ground_truth()

print("\nGROUND TRUTH COMPLETO:")
print("=" * 80)
print(f"Totale coppie: {len(ground_truth):,}")
print("\nDistribuzione label:")
print(ground_truth['label'].value_counts())
print(f"\nPercentuale match: {(ground_truth['label'].sum() / len(ground_truth) * 100):.2f}%")

print("\nCampione ground truth:")
display(ground_truth.head(10))

## 7. Visualizzazioni

In [ ]:
# Grafico distribuzione
fig, ax = plt.subplots(figsize=(6, 4))
ground_truth['label'].value_counts().plot(
    kind='bar', 
    ax=ax, 
    color=['#ff6b6b', '#51cf66']
)
ax.set_xlabel('Label')
ax.set_ylabel('Conteggio')
ax.set_title('Ground Truth Distribution (VIN-Stratified Sampling)')
ax.set_xticks([0, 1])
ax.set_xticklabels(['Non-Match', 'Match'], rotation=0)
plt.savefig('../results/visualizations/ground_truth_distribution_stratified.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Split Train/Validation/Test

In [ ]:
# Split del ground truth
print("\nCreazione split train/validation/test...")

train, validation, test = gt_generator.split_ground_truth(
    ground_truth=ground_truth,
    test_size=0.2,
    val_size=0.1,
    random_state=42
)

print("\nSTATISTICHE SPLIT:")
print("=" * 80)
print(f"Training set:   {len(train):,} coppie ({len(train)/len(ground_truth)*100:.1f}%)")
print(f"  - Match:      {train['label'].sum():,}")
print(f"  - Non-match:  {len(train) - train['label'].sum():,}")

print(f"\nValidation set: {len(validation):,} coppie ({len(validation)/len(ground_truth)*100:.1f}%)")
print(f"  - Match:      {validation['label'].sum():,}")
print(f"  - Non-match:  {len(validation) - validation['label'].sum():,}")

print(f"\nTest set:       {len(test):,} coppie ({len(test)/len(ground_truth)*100:.1f}%)")
print(f"  - Match:      {test['label'].sum():,}")
print(f"  - Non-match:  {len(test) - test['label'].sum():,}")

In [ ]:
# Visualizza distribuzione split
fig, axes = plt.subplots(1, 3, figsize=(12, 3.5))

for idx, (data, name) in enumerate([(train, 'Training'), (validation, 'Validation'), (test, 'Test')]):
    data['label'].value_counts().plot(
        kind='bar', 
        ax=axes[idx], 
        color=['#ff6b6b', '#51cf66']
    )
    axes[idx].set_title(f'{name} ({len(data):,})')
    axes[idx].set_xlabel('Label')
    axes[idx].set_xticks([0, 1])
    axes[idx].set_xticklabels(['Non-Match', 'Match'], rotation=0)

plt.savefig('../results/visualizations/ground_truth_splits_stratified.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Salvataggio Ground Truth e Dataset Campionati

In [ ]:
# Crea directory se non esiste
os.makedirs('../data/ground_truth', exist_ok=True)

# Salva i file ground truth
print("\nSalvataggio ground truth...")

train.to_csv('../data/ground_truth/train.csv', index=False)
print(f"✓ Training salvato: data/ground_truth/train.csv ({len(train):,} coppie)")

validation.to_csv('../data/ground_truth/validation.csv', index=False)
print(f"✓ Validation salvato: data/ground_truth/validation.csv ({len(validation):,} coppie)")

test.to_csv('../data/ground_truth/test.csv', index=False)
print(f"✓ Test salvato: data/ground_truth/test.csv ({len(test):,} coppie)")

ground_truth.to_csv('../data/ground_truth/ground_truth_full.csv', index=False)
print(f"✓ Ground truth completo salvato: data/ground_truth/ground_truth_full.csv ({len(ground_truth):,} coppie)")

In [ ]:
# Salva anche i dataset campionati per uso nelle fasi successive
print("\nSalvataggio dataset campionati...")

df_craigslist.to_csv('../data/processed/craigslist_sample.csv', index=False)
print(f"✓ Craigslist sample salvato: data/processed/craigslist_sample.csv ({len(df_craigslist):,} records)")

df_usedcars.to_csv('../data/processed/usedcars_sample.csv', index=False)
print(f"✓ UsedCars sample salvato: data/processed/usedcars_sample.csv ({len(df_usedcars):,} records)")

print("\n" + "="*80)
print("FASE 3 COMPLETATA CON SUCCESSO!")
print("="*80)
print(f"\n✓ Ground truth con {len(ground_truth):,} coppie ({ground_truth['label'].sum():,} match)")
print(f"✓ Dataset campionati salvati con record_id corretti (df1_XXX, df2_XXX)")
print(f"✓ Campionamento stratificato per VIN applicato")
print(f"\nProssimi passi:")
print("  → Fase 4: Rieseguire blocking strategies sui nuovi campioni")
print("  → Fase 5: Rieseguire record linkage per ottenere metriche significative")

## 10. Report Finale

In [ ]:
# Genera report
report = f"""GROUND TRUTH GENERATION REPORT (VIN-Stratified Sampling)
{'='*80}

DATASET COMPLETI:
  - Craigslist totale: {len(df_craigslist_full):,} record
  - UsedCars totale: {len(df_usedcars_full):,} record
  - VIN comuni disponibili: {len(common_vins):,}

CAMPIONAMENTO STRATIFICATO:
  - Craigslist sample: {len(df_craigslist):,} record
  - UsedCars sample: {len(df_usedcars):,} record
  - VIN comuni nel sample: {len(sample_common_vins):,}
  - Overlap percentage: {len(sample_common_vins) / (len(df_craigslist) + len(df_usedcars)) * 100:.3f}%

GROUND TRUTH:
  - Totale coppie: {len(ground_truth):,}
  - Match: {ground_truth['label'].sum():,}
  - Non-match: {(1-ground_truth['label']).sum():,}
  - Match percentage: {ground_truth['label'].mean()*100:.2f}%

SPLITS:
  - Training: {len(train):,} coppie ({train['label'].sum():,} match)
  - Validation: {len(validation):,} coppie ({validation['label'].sum():,} match)
  - Test: {len(test):,} coppie ({test['label'].sum():,} match)

FILES GENERATI:
  ✓ data/ground_truth/train.csv
  ✓ data/ground_truth/validation.csv
  ✓ data/ground_truth/test.csv
  ✓ data/ground_truth/ground_truth_full.csv
  ✓ data/processed/craigslist_sample.csv
  ✓ data/processed/usedcars_sample.csv

NOTE:
  - Campionamento stratificato applicato per massimizzare VIN overlap
  - record_id formato: df1_XXX (Craigslist), df2_XXX (UsedCars)
  - I dataset campionati DEVONO essere usati nelle Fasi 4 e 5
  - Ground truth e campioni ora allineati correttamente
"""

print(report)

# Salva report
with open('../results/ground_truth_report_stratified.txt', 'w') as f:
    f.write(report)
    
print("✓ Report salvato: results/ground_truth_report_stratified.txt")